In [ ]:
cd home/Gourav/Models

In [ ]:
print('Starting of Regressor training')
import logging
import datetime
import time
import os

import time
start_time = time.time()

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# In[10]:

print(os.getcwd())
if not os.path.exists('/home/Gourav/Models/approach_logs/Regressor/'):
    os.makedirs('/home/Gourav/Models/approach_logs/Regressor/')

In [ ]:
import time
start_time = time.time()

In [ ]:
import datetime
import numpy as np
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, udf, when
from pyspark.sql.types import *
from pyspark.sql import DataFrameStatFunctions as stat
import numpy as np
import pandas as pd
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, udf, when
from pyspark.sql.types import *
from pyspark.sql.types import *
from pyspark.sql import Window

import requests
from collections import defaultdict
from pymongo import MongoClient
from tqdm import tqdm
from pyspark.sql import DataFrameStatFunctions as stat
from pyspark.sql import Window
import pyspark.sql.functions as F
import os
import time
import json
import pickle

NIS_DATA_BASE_PATH = "gs://nis-segment-datasource-v3/processed/"
NIS_OLD_DATA_BASE_PATH = "gs://nis-localytics-datasource/processed/"

conf = SparkConf().setAll([('spark.driver.memory', '100g'), ('spark.broadcast.blockSize', '50m'), ("spark.executor.instances", '30')])
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)


In [ ]:
def get_path(dates, prefix, padding=None):
    dates_ = dates + []
    if padding:
        st_date, ed_date = sorted(dates)[0], sorted(dates)[-1]
        for i in range(1, 4):
            d = (datetime.datetime.strptime(ed_date, date_fmt) + datetime.timedelta(days=i)).strftime(date_fmt)
            if d < datetime.datetime.today().strftime(date_fmt):
                dates_.append(d)
    dates_ = list(set(dates_) - set(["2019/02/17", "2019/02/18", "2019/05/28", "2019/06/03", "2019/07/02", "2019/07/03", "2019/07/04", "2019/11/13", "2019/11/14", "2020/02/22", "2020/03/31", "2020/04/16", "2020/04/18", "2020/05/11", "2021/05/13"]))
    paths = []
    for date in dates_:
        base_path = NIS_DATA_BASE_PATH
        if date < "2018/06/26":
            base_path = NIS_OLD_DATA_BASE_PATH
        paths.append(base_path + date + "/" + prefix + "/*.parquet")
    return paths

date_fmt = "%Y/%m/%d"
month_fmt = "%Y/%m"

def millis2date(x):
    try:
        if x < 15000000000:
            return datetime.datetime.fromtimestamp(x).strftime(date_fmt)
        else:
            return datetime.datetime.fromtimestamp(x / 1000.).strftime(date_fmt)
    except:
        return "1970/01/01"

def millis2month(x):
    try:
        if x < 15000000000:
            return datetime.datetime.fromtimestamp(x).strftime(month_fmt)
        else:
            return datetime.datetime.fromtimestamp(x / 1000.).strftime(month_fmt)
    except:
        return "1970/01"

millis2date_udf = F.udf(millis2date, StringType())
millis2month_udf = F.udf(millis2month, StringType())

def divide_maps(d1, d2):
    keys = set(d1.keys()).intersection(set(d2.keys()))
    res = {}
    for k in keys:
        res[k] = d1[k] * 1. / (d2[k] + 1e-10)
    return res

def timediff(y, x, date_fmt="%Y/%m/%d"): 
    end = datetime.datetime.strptime(y, date_fmt)
    start = datetime.datetime.strptime(x, date_fmt)
    delta = (end - start).days
    return delta

def monthdiff(y, x, month_fmt="%Y/%m"): 
    millis = y - x
    delta = millis / (1000 * 3600 * 24 * 30)
    return delta

timediff_udf = udf(timediff, IntegerType())
monthdiff_udf = udf(monthdiff, IntegerType())

def filter_platform(data, platform=None):
    if platform == "ANDROID":
        data = data.filter(data.platform == "ANDROID")
    elif platform == "IOS":
        data = data.filter(data.platform != "ANDROID")
    return data

def filter_category(data, categories=None):
    if categories:
        data = data.filter(data.categoryWhenEventHappened.isin(categories))
    return data

def filter_tenant(data, tenant=None):
    if tenant in ['hi', 'HINDI']:
        data = data.filter(data.tenant.isin(['hi', 'HINDI', 'Hindi', 'hindi']))
    elif tenant in ['en', 'ENGLISH']:
        data = data.filter(~data.tenant.isin(['hi', 'HINDI', 'Hindi', 'hindi']))
    return data


def filter_app(data, app_name=None):
    if 'appName' in data.columns:
        if app_name:
            data = data.filter(data.appName == app_name)
        else:
            data = data.filter((data.appName != "mini") & (data.appName != "crux"))
    return data


# In[7]:

In [ ]:
def get_raw_path(date, hours=None):
    paths = []
    base_path = NIS_RAW_DATA_BASE_PATH + date
    if not hours:
        return base_path + "/*/*.gz"
    for hour in hours:
        paths.append(base_path + "/" + str(hour).zfill(2) + "/*.gz")
    return ",".join(paths)

def process_raw_data(paths):
    def view_data_filters(x):
        x = x['properties']
        deviceid_filter = ('deviceId' in x) and (x['deviceId'] != '')
        time_filter = ('timeSpent' in x) and (int(x['timeSpent']) <= 100) and (int(x['timeSpent']) >= 0)
        return deviceid_filter and time_filter

    try:
        rdd = sc.textFile(paths) \
            .map(json.loads) \
            .filter(lambda x: "batch" in x).flatMap(lambda x: x["batch"]) \
            .filter(lambda x: ("event" in x) and (x["event"].lower() == "timespent-front")) \
            .filter(view_data_filters) \
            .map(lambda x: x['properties'])
        view_data = rdd.map(lambda x: (x['deviceId'], x['hashId'][:-2], x['timeSpent'])) \
            .toDF(['deviceId', 'hashId', 'timeSpent'])
        
        view_data = view_data.filter(view_data.timeSpent.isNotNull())
        #view_data = view_data.filter((getHashBucketUDF(view_data.deviceId) >= 16) & (getHashBucketUDF(view_data.deviceId) <= 25))
        view_data = view_data.groupby(view_data.deviceId, view_data.hashId) \
                             .agg(F.max(view_data.timeSpent).alias('overallTimeSpent'))
        
        return view_data
    except Exception as e:
        logger.warning("Error processing data: " + str(e))


In [ ]:
from Utils import *
def getNewsData(d1, d2):
    newsMap = getNewsInDates(d1, d2)
    hashIdList = list(newsMap.keys())

    hashIdsWithFilter = []
    for h in hashIdList:
        if 'newsLanguage' in newsMap[h] and newsMap[h]['newsLanguage'] == 'english' and newsMap[h]['publishGroupList'][0]['countryCode'] == 'IN':
            hashIdsWithFilter.append(h.split('-')[0])
    
    return hashIdsWithFilter, newsMap


In [ ]:
import logging

logger = logging.getLogger(str(datetime.datetime.today().date()))
hdlr = logging.FileHandler(
    '/home/Gourav/Models/approach_logs/Regressor/' + str(datetime.datetime.today().date()) + '.log')
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr)
logger.setLevel(logging.DEBUG)

In [ ]:
def Log(s, flag=True):
    if flag:
        logger.info(s)
        print(s)

Log('----', flag=True)


In [ ]:
sampled_dates = [datetime.datetime(2023,2,1,0,0,0), datetime.datetime(2023,2,6,0,0,0),datetime.datetime(2023,2,9,0,0,0),datetime.datetime(2023,2,14,0,0,0),
                datetime.datetime(2023,2,18,0,0,0),datetime.datetime(2023,2,20,0,0,0), datetime.datetime(2023,2,25,0,0,0)]
sampled_dates

In [ ]:
train_days = 10
test_days = 7
embed_days = 30
n_days = train_days + test_days + embed_days
k = 0
st_date = (sampled_dates[k]+ datetime.timedelta(days=test_days)) - datetime.timedelta(days=n_days)


date_fmt = "%Y/%m/%d"

dates = [(st_date + datetime.timedelta(days=i)) for i in range(n_days+1)]
dates.sort()

dates_str = [date.strftime(date_fmt) for date in dates]
dates_str_embed = dates_str[0:embed_days]
dates_str_train = dates_str[embed_days:embed_days+train_days+1]
dates_str_test = dates_str[-test_days:]
print("embed data dates : ", dates_str_embed, len(dates_str_embed))
print("train data dates : ",dates_str_train, len(dates_str_train) )
print("test data dates : ",dates_str_test, len(dates_str_test))
# millisMin = dates[0].timestamp() * 1000
# millisMax = (dates[-1] + datetime.timedelta(days=1)).timestamp() * 1000
print(dates[0], dates[-1])
hashIdsWithFilter, newsMap = getNewsData(dates[0], dates[-1])

In [ ]:
#need this for live data else skip if backtesting
NIS_RAW_DATA_BASE_PATH = "gs://inshorts-segment-raw/data/segment-raw-v5a/"

path = get_raw_path(datetime.datetime.now().strftime("%Y/%m/%d"))
today_data = process_raw_data(path)
today_data = today_data.filter(today_data.hashId.isin(hashIdsWithFilter))

In [ ]:
#generate datasets
def generate_data(datestr):
#     datestr = [d.strftime(date_fmt) for d in dates]
    datestr = [d.strftime(date_fmt) for d in dates]
    paths = get_path(datestr, 'timeSpentFrontEvents')

    data = sqlContext.read.parquet(*paths)

    data = filter_app(data, app_name=None)
    data = filter_tenant(data, tenant='en')

    # data = data.filter((data.eventTimestamp > millisMin) & (data.eventTimestamp < millisMax))
    data = data.filter(data.categories.isNotNull())
    from pyspark.sql.functions import explode

    #data=data.select(data.deviceId, (F.split(data.hashId, '-')[0]).alias('hashId'),data.notificationOpened,explode(data.categories)).withColumnRenamed("col","category")
    data = data.select(data.deviceId, data.overallTimeSpent, (F.split(data.hashId, '-')[0]).alias('hashId')).groupby('deviceId', 'hashId').agg(F.max('overallTimeSpent').alias('overallTimeSpent'))

    data = data.filter(data.hashId.isin(hashIdsWithFilter))
    return data

data_embed = generate_data(dates_str_embed)
data_train = generate_data(dates_str_train)
data_test = generate_data(dates_str_test)

In [ ]:
data_test

In [ ]:
#to be used when live data is needed
data = data.union(today_data)
data.cache()

In [ ]:
def popularity(data, newsMeanMap):
    Log("Computing news mean")
    newsMeandf = data.groupby(data.hashId).agg({'overallTimeSpent' : 'sum', 'hashId' : 'count'}).toPandas()

    for v in newsMeandf.values:
        newsMeanMap[v[0]] = newsMeanMap[v[0]][0] + v[1],  newsMeanMap[v[0]][1] + v[2]

newsMeanMap = defaultdict(lambda: (7, 1))
popularity(data_embed, newsMeanMap)

data_embed = data_embed.filter(F.udf(lambda hashId, overallTimeSpent: overallTimeSpent > (2.5 * newsMeanMap[hashId][0]/newsMeanMap[hashId][1]), BooleanType())('hashId', 'overallTimeSpent'))

In [ ]:
def popularity(data, newsMeanMap):
    Log("Computing news mean")
    newsMeandf = data.groupby(data.hashId).agg({'overallTimeSpent' : 'sum', 'hashId' : 'count'}).toPandas()

    for v in newsMeandf.values:
        newsMeanMap[v[0]] = newsMeanMap[v[0]][0] + v[1],  newsMeanMap[v[0]][1] + v[2]

newsMeanMap = defaultdict(lambda: (7, 1))
popularity(data_train, newsMeanMap)

data_train = data_train.filter(F.udf(lambda hashId, overallTimeSpent: overallTimeSpent > (2.5 * newsMeanMap[hashId][0]/newsMeanMap[hashId][1]), BooleanType())('hashId', 'overallTimeSpent'))

In [ ]:
def popularity(data, newsMeanMap):
    Log("Computing news mean")
    newsMeandf = data.groupby(data.hashId).agg({'overallTimeSpent' : 'sum', 'hashId' : 'count'}).toPandas()

    for v in newsMeandf.values:
        newsMeanMap[v[0]] = newsMeanMap[v[0]][0] + v[1],  newsMeanMap[v[0]][1] + v[2]

newsMeanMap = defaultdict(lambda: (7, 1))
popularity(data_test, newsMeanMap)

data_test = data_test.filter(F.udf(lambda hashId, overallTimeSpent: overallTimeSpent > (2.5 * newsMeanMap[hashId][0]/newsMeanMap[hashId][1]), BooleanType())('hashId', 'overallTimeSpent'))

In [ ]:
data_embed.printSchema()

In [ ]:
data_comb = data_embed.union(data_train)
#data_comb.groupBy('eventName').count().orderBy('count').show()

In [ ]:
data_comb

In [ ]:
data_comb.filter(data_comb.notificationOpened > 0).groupBy('eventName').count().orderBy('count').show()

In [ ]:
# whats happening in next few rowS?

In [ ]:
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.clustering import LDA
import time

training_start = time.time()


# data_filtered = data.filter(F.udf(lambda hashId, noti: overallTimeSpent > (2.5 * newsMeanMap[hashId][0]/newsMeanMap[hashId][1]), BooleanType())('hashId', 'overallTimeSpent'))
# abcset=sqlContext.read.parquet("gs://pvtrough_asia_south1/tf_idf/devices_fulfilling_criteria")
# df_temp=abcset.filter(abcset.set_typ == 'roberta').select(['deviceid']).distinct()
# df_temp=df_temp.withColumnRenamed("deviceid","deviceId")

# data_filtered=data_filtered.join(df_temp,["deviceId"],"inner")

data_embed = data_embed.filter(data_embed.notiOpened>0)    


# Log("Training took %s minutes"%(int((time.time() - training_start) / 60)))

In [ ]:
d1 = data.select('deviceId').distinct().collect()
d2 = data_filtered.select('deviceId').distinct().collect()
d3 = df_temp.select('deviceId').distinct().collect()

In [ ]:
len(d2)

In [ ]:
from tqdm import tqdm
df_left = pd.DataFrame(columns = ['deviceId', 'cluster_left_out_date'])
left_out = set(d3) - set(d2)
pbar = tqdm(total=len(left_out))
cluster_left_out = (datetime.datetime.now() + datetime.timedelta(days=0)) .strftime("%Y_%m_%d") 
i = 0
for d in left_out:
    df_left.loc[i,'deviceId'] = d['deviceId']
    df_left.loc[i,'cluster_left_out_date'] = cluster_left_out
    i= i+1
    pbar.update(1)
pbar.close()

In [ ]:
dfl = df_left.copy()
print(dfl.head())

In [ ]:

path_leftout = 'gs://pvtrough_asia_south1/clustering/leftout_devices_roberta'
df_left = sqlContext.createDataFrame(dfl[['deviceId', 'cluster_left_out_date']])
df_left.coalesce(1).write.partitionBy('cluster_left_out_date').csv(path_leftout , sep=',', header=True,mode = 'append')

    

In [ ]:
#till here

In [ ]:
import torch.nn as nn
from torch.nn.parallel import DataParallel
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer , RobertaForSequenceClassification
from transformers import EarlyStoppingCallback


model_name = "roberta-base"
num_classes = 17
# model_path = "output/checkpoint-12000"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_classes,output_hidden_states=True)

# Move the model to the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Check the number of available GPUs
if torch.cuda.device_count() > 1:
    # Specify which GPUs to use
    device_ids = [0, 1]  # Adjust the GPU IDs based on your system configuration
    model = DataParallel(model, device_ids=device_ids)
    print(isinstance(model, DataParallel))


In [ ]:
def get_embedding(model, tokenizer, text):
    model.eval()

    with torch.no_grad():
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        input_ids = tokenizer.encode(text, add_special_tokens=True, return_tensors='pt').to(device)
        outputs = model(input_ids=input_ids)
        hidden_states = outputs.hidden_states
        last_layer_hidden_states = hidden_states[-1]
        embeddings = torch.mean(last_layer_hidden_states, dim=1).squeeze().cpu().numpy()

    return embeddings

In [ ]:
import os
import sys
import torch
import numpy as np

from transformers import RobertaModel, RobertaTokenizer

from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity


def Dim_reduction(sentences, tokenizer, model):
    '''
        This method will accept array of sentences, roberta tokenizer & model
        next it will call methods for dimention reduction
    '''

    vecs = []
    with torch.no_grad():

        for sentence in sentences:
            inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True,  max_length=64)
            inputs['input_ids'] = inputs['input_ids'].to(DEVICE)
            inputs['attention_mask'] = inputs['attention_mask'].to(DEVICE)

            hidden_states = model(**inputs, return_dict=True, output_hidden_states=True).hidden_states

            #Averaging the first & last hidden states
            output_hidden_state = (hidden_states[-1] + hidden_states[1]).mean(dim=1)

            vec = output_hidden_state.cpu().numpy()[0]

            vecs.append(vec)
    
#     print("35:",vecs)
    #Finding Kernal
    kernel, bias = compute_kernel_bias([vecs])
    kernel = kernel[:, :200]
    #If you want to reduce it to 128 dim
    #kernel = kernel[:, :128]
    embeddings = []
    embeddings = np.vstack(vecs)
#     print("43:", kernel, bias)
#     print("44:",embeddings)
    #Sentence embeddings can be converted into an identity matrix
    #by utilizing the transformation matrix
    embeddings = transform_and_normalize(embeddings, 
                kernel=kernel,
                bias=bias
            )
    return embeddings

In [ ]:
import numpy as np

def transform_and_normalize(vecs, kernel, bias):
    """
        Applying transformation then standardize
    """
    if not (kernel is None or bias is None):
        vecs = (vecs + bias).dot(kernel)
    return normalize(vecs)
    
def normalize(vecs):
    """
        Standardization
    """
    return vecs / (vecs**2).sum(axis=1, keepdims=True)**0.5
    
def compute_kernel_bias(vecs):
    """
    Calculate Kernal & Bias for the final transformation - y = (x + bias).dot(kernel)
    """
    vecs = np.concatenate(vecs, axis=0)
    mu = vecs.mean(axis=0, keepdims=True)
    cov = np.cov(vecs.T)
    u, s, vh = np.linalg.svd(cov)
    W = np.dot(u, np.diag(s**0.5))
    W = np.linalg.inv(W.T)
    return W, -mu

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# tokenizer = RobertaTokenizer.from_pretrained('C:/Models/roberta-base-nli-stsb-mean-tokens')
# model = RobertaModel.from_pretrained('C:/Models/roberta-base-nli-stsb-mean-tokens')

sentences = ['someone is slicing a onion', 'someone is carrying a fish']

embeddings = Dim_reduction(sentences, tokenizer, model)
# print(distance.cosine(embeddings[0],embeddings[1]))
print(f'Similarity score of two sentences {cosine_similarity([embeddings[0],embeddings[1]])[0][1]}')
print(f"New Dimension is {np.shape(embeddings)}")

In [ ]:
embeddings[0]

# Checking for News embeddings just using title

In [ ]:
# reducing the size of embeddings using dimensionally reduction

In [ ]:
k=0
from pyspark.sql.functions import array, col
path_embedding = "gs://pvtrough_asia_south1/clustering/News_tilte_embedding"+sampled_dates[k].strftime("%Y_%m_%d")+"/"
df_embedding = sqlContext.read.csv(path_embedding, sep=',',
                         inferSchema=True, header=True)
# df_embedding.take(1)

In [ ]:
#generate news embeddings based on notification text
embedding_start = time.time()
document_embeddings_title={}

newsMapProcessed = {}
document_embeddings = {}
#try:
#    already_embedded = df_embedding.select('hid').distinct().collect()
#    already_embedded = [x['hid'] for x in already_embedded]
#except:
#    print("News Embeddings not found, generating new one")
already_embedded = []
for hId in tqdm(newsMap):
    
    h = hId.split('-')[0]
#     print(h)
    if h in already_embedded or (h not in hashIdsWithFilter) :
        continue
    else:    
        newsMapProcessed[h] = {}
        newsMapProcessed[h]['title'] = newsMap[hId]['title']
        newsMapProcessed[h]['content'] = newsMap[hId]['content']
        newsMapProcessed[h]['features'] = newsMapProcessed[h]['title'] #+ "." + newsMapProcessed[h]['content']
        document_embeddings[h] = get_embedding(model, tokenizer, newsMapProcessed[h]['features'])
        

        document_embeddings_title[h] = document_embeddings[h]
        

Log("Embedding took %s minutes"%(int((time.time() - embedding_start) / 60)))
    

In [ ]:
len(document_embeddings_title)

In [ ]:
path_embedding = "gs://pvtrough_asia_south1/clustering/News_tilte_embedding"+sampled_dates[k].strftime("%Y_%m_%d")+"/"


In [ ]:
if len(document_embeddings_title) > 0:
    df_em_pandas = pd.DataFrame(document_embeddings_title).T
    df_em_pandas['hid'] = df_em_pandas.index
    df_em_pandas['embed_date'] = sampled_dates[0]
    df_em_spark = (sqlContext.createDataFrame(df_em_pandas))
    df_em_spark.write.partitionBy('embed_date').csv(path_embedding , sep=',', header=True,mode = 'overwrite')

# News Embeddings using content and title

In [ ]:
from pyspark.sql.functions import array, col

path_embedding = "gs://pvtrough_asia_south1/clustering/Content_news_embedding.csv"+sampled_dates[k].strftime("%Y_%m_%d")+"/"

df_news_embedding = sqlContext.read.csv(path_embedding, sep=',',
                         inferSchema=True, header=True)
df_news_embedding = df_news_embedding.distinct()
feat_cols = [str(x) for x in range(0,len(df_news_embedding.columns) - 2)]
df_embedding_exploded = df_news_embedding.distinct()
df_news_embedding = df_news_embedding.select('hid', array([col(x) for x in feat_cols ]).alias('embedding'))
df_news_embedding.take(1)


In [ ]:
df_news_embedding.count()

In [ ]:
df_news_embedding.select(['hid']).distinct().count()

# Checking for deviceids embeddings using content and title

In [ ]:

path_embedding = "gs://pvtrough_asia_south1/clustering/Content_news_embedding.csv"+sampled_dates[k].strftime("%Y_%m_%d")+"/"


In [ ]:
from pyspark.sql.functions import array, col
path_embedding = 'gs://pvtrough_asia_south1/clustering/Content_news_embedding.csv'
df_embedding = sqlContext.read.csv(path_embedding, sep=',',
                         inferSchema=True, header=True)


# df_embedding.take(1)

In [ ]:
#generate news embeddings based on notification text
newsMapProcessed = {}
document_embeddings_content = {}
document_embeddings={}
embedding_start = time.time()

#try:
#    already_embedded = df_embedding.select('hid').distinct().collect()
#    already_embedded = [x['hid'] for x in already_embedded]
#except:
#    print("News Embeddings not found, generating new one")
already_embedded = []
for hId in tqdm(newsMap):
    
    h = hId.split('-')[0]
#     print(h)
    if h in already_embedded or (h not in hashIdsWithFilter) :
        continue
    else:    
        newsMapProcessed[h] = {}
        newsMapProcessed[h]['title'] = newsMap[hId]['title']
        newsMapProcessed[h]['content'] = newsMap[hId]['content']
        newsMapProcessed[h]['features'] = newsMapProcessed[h]['title'] + newsMapProcessed[h]['content']
        document_embeddings[h] = get_embedding(model, tokenizer, newsMapProcessed[h]['features'])

        document_embeddings_content[h] = document_embeddings[h]

        

        

Log("Embedding took %s minutes"%(int((time.time() - embedding_start) / 60)))
     

In [ ]:
len(document_embeddings_content['l_ebpzzatk'])

In [ ]:
len(document_embeddings_content)

In [ ]:
#generate news embeddings based on notification text
newsMapProcessed = {}
document_embeddings = {}
try:
    already_embedded = df_embedding.select('hid').distinct().collect()
    already_embedded = [x['hid'] for x in already_embedded]
except:
    print("News Embeddings not found, generating new one")
    already_embedded = []
for hId in tqdm(newsMap):
    
    h = hId.split('-')[0]
#     print(h)
    if h in already_embedded or (h not in hashIdsWithFilter) :
        continue
    else:    
        newsMapProcessed[h] = {}
        newsMapProcessed[h]['title'] = newsMap[hId]['title']
        newsMapProcessed[h]['content'] = newsMap[hId]['content']
        newsMapProcessed[h]['features'] = newsMapProcessed[h]['title'] + "." + newsMapProcessed[h]['content']
        document_embeddings[h] = get_embedding(model, tokenizer, newsMapProcessed[h]['features'])
     

In [ ]:
path_embedding = "gs://pvtrough_asia_south1/clustering/Content_news_embedding.csv"+sampled_dates[k].strftime("%Y_%m_%d")+"/"


In [ ]:
if len(document_embeddings_content) > 0:
    df_em_pandas = pd.DataFrame(document_embeddings_content).T
    df_em_pandas['hid'] = df_em_pandas.index
    df_em_pandas['embed_date'] = sampled_dates[0]
    df_em_spark = (sqlContext.createDataFrame(df_em_pandas))
    
    df_em_spark.write.partitionBy('embed_date').csv(path_embedding , sep=',', header=True,mode = 'overwrite')

In [ ]:
df_embedding = sqlContext.read.csv(path_embedding, sep=',',
                         inferSchema=True, header=True)

from pyspark.sql.functions import array, col
feat_cols = [str(x) for x in range(0,len(df_embedding.columns) - 2)]
df_embedding = df_embedding.select('hid', array([col(x) for x in feat_cols ]).alias('embedding'))
df_embedding.take(1)

#df_embedding=df_embedding.withColumnRenamed("hid","hashId")

In [ ]:
df_embedding.count()

In [ ]:
from pyspark.sql.functions import *

k = 0
data_filtered = data_embed
# # calculate idf_weight
N = data_filtered.select('hashId').distinct().count()
users = data_filtered.groupBy('hashId').agg(F.countDistinct('deviceId').alias('users'))
idf_weights = users.withColumn('idf_weight', F.log(N / F.col('users')))

# Multiply embeddings with IDF weights
df_result = df_embedding.join(idf_weights, df_embedding.hid == idf_weights.hashId)
df_result = df_result.withColumn('weighted_embedding', F.expr('transform(embedding, (x, i) -> x * idf_weight)'))


df_result = df_result.select('hid','weighted_embedding')
df_joined = data_filtered.join(df_result, data_filtered.hashId == df_result.hid , 'left')


# assuming your dataframe is called df
grouped_df = df_joined.groupBy("deviceId").agg(*[
    expr(f"avg(weighted_embedding[{i}]) as avg_{i}") for i in range(768)
])



# assuming your dataframe is called grouped_df
avg_cols = [col(f"avg_{i}") for i in range(768)]
grouped_df = grouped_df.withColumn("topic_avg_embedding", array(*avg_cols))

path_vec = "gs://pvtrough_asia_south1/clustering/MLP_user_embeddings"+sampled_dates[k].strftime("%Y_%m_%d")+"/"
path_vec_exploded = "gs://pvtrough_asia_south1/clustering/MLP_device_vectors_exploded_"+sampled_dates[k].strftime("%Y_%m_%d")+"/"
print(path_vec, path_vec_exploded)

grouped_df.select('deviceId', 'topic_avg_embedding').write.parquet(path=path_vec, mode='overwrite')
#grouped_df.select('deviceId', *avg_cols).write.parquet(path=path_vec_exploded, mode='overwrite')


In [ ]:
grouped_df.take(1)

In [ ]:
from pyspark.sql.functions import array_contains, size, col
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

k = 0
#num_clusters = 1000
path_vec = "gs://pvtrough_asia_south1/clustering/MLP_user_embeddings"+sampled_dates[k].strftime("%Y_%m_%d")+"/"
# path_vec = "gs://pvtrough_asia_south1/clustering/device_vectors_roberta_live/"
print(path_vec)
# print(path_vec)

temp = sqlContext.read.parquet(path_vec)
df_devices_embeddings = temp.filter(~array_contains(temp.topic_avg_embedding, float('nan')))


df_devices_embeddings = df_devices_embeddings.dropna(how='all')
df_devices_embeddings = df_devices_embeddings.where(size(col("topic_avg_embedding")) == 768)







In [ ]:
df_devices_embeddings.take(1)

In [ ]:
df_news_embedding = df_news_embedding.where(size(col("embedding")) == 768)


In [ ]:
train_data = data_train.join(df_devices_embeddings,["deviceId"],"inner")
train_data = train_data.join(df_news_embedding,train_data.hashId == df_news_embedding.hid,"inner")
train_data = train_data.select('deviceId', 'hashId', 'topic_avg_embedding', 'embedding','overallTimeSpent')
#train_data = train_data.withColumn("notiOpened",F.when(train_data.notiOpened > 1, 
#                                                     1).otherwise(train_data.notiOpened))
train_data.printSchema()

In [ ]:
from pyspark.sql.functions import col, concat

train_data_features = train_data.withColumn('embedding_features', 
                                            concat(col('topic_avg_embedding'), col('embedding')))\
                                .select('embedding_features', 'overallTimeSpent')
                                    

In [ ]:
train_data_features.take(1)

In [ ]:
from pyspark.ml.functions import array_to_vector

train_data_features=train_data_features.select(array_to_vector('embedding_features').alias('vec_embedding'),'overallTimeSpent')#.collect()

In [ ]:
train_data_features.take(1)

In [ ]:
train_data_features.count()#70973387

In [ ]:
train_data_features

In [ ]:
path_vec=path_vec = "gs://pvtrough_asia_south1/clustering/Regressor_train"+sampled_dates[k].strftime("%Y_%m_%d")+"/"

train_data_features.select('vec_embedding', 'overallTimeSpent').write.parquet(path=path_vec, mode='overwrite')


# Training

In [ ]:
from pyspark.ml.regression import LinearRegression

training_start = time.time()
data = train_data_features.select("vec_embedding", "overallTimeSpent")#.sample(fraction = 0.01)
(training_data, testing_data) = data.randomSplit([0.7, 0.3])#.sample(fraction)

lr = LinearRegression(featuresCol = 'vec_embedding', labelCol='overallTimeSpent')
lr_model = lr.fit(training_data)

#Log("Coefficients: " + str(lr_model.coefficients))
Log("Intercept: " + str(lr_model.intercept))

trainingSummary = lr_model.summary
#Log("RMSE_train: %f" % trainingSummary.rootMeanSquaredError)
#Log("r2: %f" % trainingSummary.r2)

lr_predictions = lr_model.transform(training_data)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="overallTimeSpent",metricName="r2")
Log("R Squared (R2) on train data = %g" % lr_evaluator.evaluate(lr_predictions))
train_result = lr_model.evaluate(training_data)
Log("Root Mean Squared Error (RMSE) on train data = %g" % train_result.rootMeanSquaredError)



lr_predictions = lr_model.transform(testing_data)
#lr_predictions.select("prediction","overallTimeSpent","vec_embedding").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="overallTimeSpent",metricName="r2")
Log("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

test_result = lr_model.evaluate(testing_data)
Log("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Log("numIterations: %d" % trainingSummary.totalIterations)
#Log("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
Log("Model training took %s minutes"%(int((time.time() - training_start) / 60)))


In [ ]:
lr_predictions.show()

23/06/25 16:45:07 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1053.4 KiB


# Testing

In [ ]:
# try to use DL here also, will try to use Pytorch